# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770885094566                   -0.52    9.0         
  2   -2.772148134110       -2.90       -1.32    1.0    105ms
  3   -2.772170186930       -4.66       -2.43    1.0    106ms
  4   -2.772170649240       -6.34       -3.15    1.0    122ms
  5   -2.772170722798       -7.13       -4.40    2.0    122ms
  6   -2.772170722927       -9.89       -4.63    1.0    133ms
  7   -2.772170723013      -10.07       -5.50    1.0    113ms
  8   -2.772170723014      -12.14       -5.57    2.0    130ms
  9   -2.772170723015      -11.96       -6.90    1.0    126ms
 10   -2.772170723015      -14.05       -6.83    2.0    140ms
 11   -2.772170723015      -14.65       -7.40    1.0    134ms
 12   -2.772170723015      -14.45       -8.00    2.0    132ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.7735583557305457

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770730642013                   -0.53    9.0         
  2   -2.772053325527       -2.88       -1.30    1.0    140ms
  3   -2.772082912936       -4.53       -2.64    1.0    106ms
  4   -2.772083416557       -6.30       -4.11    2.0    122ms
  5   -2.772083417787       -8.91       -4.87    2.0    163ms
  6   -2.772083417809      -10.65       -5.78    1.0    110ms
  7   -2.772083417811      -11.92       -6.35    2.0    126ms
  8   -2.772083417811      -13.65       -6.84    1.0    134ms
  9   -2.772083417811      -14.88       -8.20    2.0    128ms

Polarizability via ForwardDiff:       1.772534973943165
Polarizability via finite difference: 1.7735583557305457
